# FastText - Algebra Problems (Anirudh A)
### Load Dataset, Make Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import nltk
nltk.download('stopwords')
import re
from bs4 import BeautifulSoup
import numpy as np
import torch

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
import logging
import gensim
import csv

In [4]:
!pip install scipy

In [5]:
path = "/content/drive/MyDrive/wiki-news-300d-1M.vec"
en_model = KeyedVectors.load_word2vec_format(path)

In [6]:
!pip install transformers
import transformers as ppb 

In [7]:
import pandas as pd
path = "/content/drive/MyDrive/MWP(Words and Values in List).csv"
df = pd.read_csv(path)
df.head()

,Unnamed: 0,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


### Data Cleaning 

In [8]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS =nltk.corpus.stopwords.words('english')

In [9]:
def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

### FastText

In [10]:
a=en_model['king'] #Example
a

array([ 1.082e-01,  4.450e-02, -3.840e-02,  1.100e-03, -8.880e-02,
        7.130e-02, -6.960e-02, -4.770e-02,  7.100e-03, -4.080e-02,
       -7.070e-02, -2.660e-02,  5.000e-02, -8.240e-02,  8.480e-02,
       -1.627e-01, -8.510e-02, -2.950e-02,  1.534e-01, -1.828e-01,
       -2.208e-01,  2.430e-02, -9.210e-02, -1.089e-01, -1.009e-01,
       -1.190e-02,  3.770e-02,  2.038e-01,  7.200e-02,  2.020e-02,
        2.798e-01,  1.150e-02, -1.510e-02,  1.037e-01,  4.000e-04,
       -1.040e-02,  1.960e-02,  1.265e-01,  8.280e-02, -1.369e-01,
        1.070e-01,  1.270e-01, -3.490e-02, -6.830e-02, -1.140e-02,
        3.370e-02,  1.260e-02,  7.920e-02,  4.400e-02, -2.530e-02,
        4.890e-02, -7.850e-02, -6.259e-01, -9.720e-02,  1.654e-01,
       -5.780e-02, -4.370e-02,  4.090e-02, -1.820e-02, -1.891e-01,
        2.770e-02, -1.460e-02, -5.310e-02,  4.260e-02,  4.900e-03,
        4.000e-03,  1.423e-01, -9.750e-02, -3.500e-03,  9.630e-02,
       -1.900e-03, -1.466e-01, -1.662e-01,  6.650e-02, -1.500e

In [11]:
def sent_vectorizer(sent, model):
  sent_vec =[]
  numw = 0
  for w in sent:
    try:
      if numw == 0:
        sent_vec = model[w]
      else:
        sent_vec = np.add(sent_vec, model[w])
      numw+=1
    except:
      pass
  return np.asarray(sent_vec) / numw



In [12]:
df.iloc[:,1] = df.iloc[:,1].apply(clean_text)
print(df.iloc[:100,1])

0     constants alpha beta frac x alpha x+ beta frac...
1     length segment points 2a a4 4 1 2 sqrt 10 unit...
2     chewbacca 20 pieces cherry gum 30 pieces grape...
3     constants b let f x left begin array cl ax + b...
4             calculate sum 1 + 3 + 5 + cdots + 15 + 17
                            ...                        
95    largest value x satisfies equation sqrt 2x 4x ...
96    sum left dfrac 1 3 right + left dfrac 1 3 righ...
97    map 12centimeter length represents 72 kilomete...
98                      find sum values x 2 x23x2 4 x 4
99              given 0 f g 8 f x x2 + 8 g x x2 4 value
Name: Problem , Length: 100, dtype: object


In [13]:
d = [];
for sent in df.iloc[:,1]:
  d.append(sent_vectorizer(sent, en_model))

In [14]:
df1 = pd.DataFrame(d)
df1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-0.033060,0.012716,-0.066499,0.003662,0.030556,0.066100,0.002122,-0.045769,0.100774,0.020540,0.003119,-0.043901,0.049941,0.013481,0.015846,0.055594,-0.017490,0.025826,-0.023179,0.052326,0.023097,-0.130174,0.016110,-0.028276,-0.058068,-0.005160,0.039490,0.135469,0.020103,-0.068572,0.021159,0.035191,-0.003069,-0.021797,0.018375,0.004572,-0.006446,0.010003,0.066546,0.007509,...,0.003213,-0.002143,-0.022934,0.029262,-0.214254,0.016754,0.025165,0.002526,0.032685,-0.015460,0.002853,0.034788,0.012566,0.043834,0.012521,0.023793,-0.080284,-0.080553,0.026315,0.034178,0.022269,0.109729,0.005418,-0.022737,0.015379,-0.040534,0.100651,0.007749,0.003031,0.048259,-0.026588,0.036143,-0.010381,0.048068,0.038351,0.033554,-0.008644,-0.041012,0.024126,-0.003468
1,-0.101459,0.011697,-0.081979,-0.032010,0.048881,0.061566,0.011028,-0.041976,0.118157,0.023731,-0.024581,-0.039938,0.043259,-0.022140,-0.021141,0.048457,-0.009671,-0.008705,0.003395,0.089262,0.008786,-0.146443,-0.031900,-0.024614,-0.092716,-0.040516,0.052102,0.165233,0.055653,-0.088369,0.035067,0.010969,0.009700,-0.021340,0.078366,0.030166,-0.007681,0.032590,0.045247,-0.009571,...,0.010497,-0.016057,-0.003066,0.026505,-0.285426,0.032012,0.056640,-0.019853,0.050302,-0.024036,-0.000457,0.029474,0.018674,0.066436,-0.004560,-0.001376,-0.090407,-0.109616,0.045978,0.023588,0.002541,0.086752,-0.000679,-0.035624,-0.003041,-0.038903,0.119217,-0.004662,-0.001003,0.039547,0.005466,0.016448,0.007195,0.060026,0.033598,0.027974,0.020221,-0.020988,-0.017419,0.010876
2,-0.091296,0.029833,-0.096026,-0.049331,0.049579,0.051831,0.010815,-0.053386,0.141332,0.032405,-0.016584,-0.038277,0.053828,-0.035082,-0.034811,0.057767,-0.019344,-0.044332,-0.021840,0.108087,0.002107,-0.140586,-0.025884,-0.015824,-0.088974,-0.074600,0.027448,0.173239,0.043674,-0.108042,0.040582,0.052190,-0.009247,-0.042035,0.074126,0.047275,-0.000460,0.043591,0.065654,-0.013246,...,0.013791,-0.027740,0.007475,0.014177,-0.321502,0.073183,0.049807,-0.024530,0.032729,-0.027436,-0.025004,0.024817,0.033022,0.061907,-0.029352,0.002212,-0.108666,-0.103559,0.043361,0.048481,0.023762,0.075788,0.006216,-0.029184,0.038546,-0.052259,0.172130,-0.003063,0.005931,0.023898,0.022366,0.017215,-0.009071,0.061168,0.018065,0.037406,-0.008278,-0.042560,-0.023389,0.003145
3,-0.066093,0.019570,-0.102819,-0.028517,0.008690,0.052083,-0.011769,-0.064647,0.135179,0.050548,-0.001637,-0.077104,0.042167,-0.014062,-0.006497,0.072391,-0.028374,-0.002948,-0.021154,0.072558,-0.000597,-0.171696,-0.008392,-0.024493,-0.094093,-0.056593,0.029136,0.186218,0.024578,-0.096364,0.043785,0.043573,0.016563,-0.037178,0.045534,0.043531,-0.011464,0.015387,0.098797,-0.001735,...,0.004129,-0.001579,-0.029289,0.021407,-0.284922,0.061069,0.064145,0.002590,0.065324,-0.020008,-0.010930,0.048726,0.014657,0.041447,-0.016356,0.024251,-0.099843,-0.100953,0.043918,0.055879,0.005967,0.114436,-0.022218,-0.027037,0.019749,-0.039922,0.167807,0.014118,0.026609,0.040090,-0.005115,0.041151,-0.017740,0.058753,0.052343,0.040280,-0.029016,-0.045978,-0.023160,0.010299
4,-0.038310,0.019421,-0.026648,0.002886,0.068855,0.084076,0.016979,-0.048769,0.113452,0.015186,-0.025362,-0.024607,0.051134,0.018531,-0.001772,0.069869,-0.005969,-0.012393,0.008597,0.061897,0.029390,-0.106528,0.017183,-0.030321,-0.063776,0.006783,0.043931,0.145445,0.049679,-0.090855,0.014155,0.023572,-0.028607,-0.008641,0.025110,0.006521,0.029883,0.014800,0.030038,0.016993,...,0.008241,-0.022610,-0.012559,0.027045,-0.204472,-0.003048,-0.006390,-0.035445,0.068545,-0.007783,0.011034,0.025510,0.009338,0.028490,0.036424,0.034066,-0.073355,-0.065438,0.004297,0.017776,-0.017472,0.090300,0.020455,-0.020234,-0.000531,-0.025717,0.079562,-0.001203,-0.020483,0.052869,-0.011803,0.048976

In [15]:
from google.colab import files
df1.to_csv('FastText_algebra.csv') 
files.download('FastText_algebra.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>